## Bribes research

In [18]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.constants import BRIBES_URL
from notebooks.bribes_research.constants import DASHBOARD_URL
from notebooks.bribes_research.constants import PROTOCOL_AURA
from notebooks.bribes_research.constants import PROTOCOL_CVX
from notebooks.bribes_research.constants import TABLE_COLUMNS
from notebooks.bribes_research.helpers import calculate_metrics
from notebooks.bribes_research.helpers import fetch_token_prices

pd.set_option('display.max_rows', 500)
BADGER_BRIBE = 42000
STEP = 2000
BADGER_LOW_BOUND = 2000
BADGER_UPPER_BOUND = BADGER_BRIBE

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()
total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']
for bribe in range(BADGER_LOW_BOUND, BADGER_UPPER_BOUND, STEP):
    badger_bribe_in_dollar = bribe * badger_price
    # cvx_dilution_100, cvx_in_out_100 = calculate_metrics(badger_bribe_in_dollar, total_bribes_cvx,
    #                                                      dollar_spent_on_bribes_cvx)
    # table_rows.append(
    #     [PROTOCOL_CVX, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar, cvx_dilution_100, cvx_in_out_100]
    # )
    # aura_dilution_100, aura_in_out_100 = calculate_metrics(badger_bribe_in_dollar, total_bribes_aura,
    #                                                        dollar_spent_on_bribes_aura)
    # table_rows.append(
    #     [PROTOCOL_AURA, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar, aura_dilution_100, aura_in_out_100]
    # )
    cvx_dilution_50, cvx_in_out_50 = calculate_metrics(badger_bribe_in_dollar / 2, total_bribes_cvx,
                                                       dollar_spent_on_bribes_cvx)
    aura_dilution_50, aura_in_out_50 = calculate_metrics(badger_bribe_in_dollar / 2, total_bribes_aura,
                                                         dollar_spent_on_bribes_aura)
    # table_rows.append(
    #     [PROTOCOL_AURA, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100,
    #      badger_bribe_in_dollar / 2, aura_dilution_50, aura_in_out_50]
    # )

    # Rebalance here
    cvx_in_out_floating = cvx_in_out_50
    aura_in_out_floating = aura_in_out_50
    rebalancing_step = 0.01  # in %
    cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar / 2
    cvx_dilution = None
    aura_dilution = None
    while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
        if cvx_in_out_floating > aura_in_out_floating:
            # Take bribe portion from aura and give it to cvx
            bribe_stolen = aura_bribe_portion * rebalancing_step
            aura_bribe_portion -= bribe_stolen
            cvx_bribe_portion += bribe_stolen
        elif aura_in_out_floating > cvx_in_out_floating:
            # Take bribe portion from cvx and give it to aura
            bribe_stolen = aura_bribe_portion * rebalancing_step
            cvx_bribe_portion -= bribe_stolen
            aura_bribe_portion += bribe_stolen
        if cvx_bribe_portion <= 0 or aura_bribe_portion <= 0:
            break
        cvx_dilution, cvx_in_out_floating = calculate_metrics(cvx_bribe_portion, total_bribes_cvx,
                                                              dollar_spent_on_bribes_cvx)
        aura_dilution, aura_in_out_floating = calculate_metrics(aura_bribe_portion, total_bribes_aura,
                                                                dollar_spent_on_bribes_aura)

    table_rows.append(
        [PROTOCOL_CVX, badger_bribe_in_dollar, round(cvx_bribe_portion / badger_bribe_in_dollar * 100, 2),
         cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
    )
    table_rows.append(
        [PROTOCOL_AURA, badger_bribe_in_dollar, round(aura_bribe_portion / badger_bribe_in_dollar * 100, 2),
         aura_bribe_portion, aura_dilution, aura_in_out_floating]
    )
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

,Protocol,BADGER total bribe in $,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,8560.0,-0.34,-28.947217,0.002082,149.538581
1,AURA,8560.0,100.34,8588.947217,2.711081,247.349680
2,CVX,17120.0,-0.34,-57.894433,0.004163,149.535468
3,AURA,17120.0,100.34,17177.894433,5.422161,240.456964
4,CVX,25680.0,-0.34,-86.841650,0.006245,149.532355
5,AURA,25680.0,100.34,25766.841650,8.133242,233.564247
6,CVX,34240.0,-0.34,-115.788867,0.008326,149.529242
7,AURA,34240.0,100.34,34355.788867,10.844323,226.671531
8,CVX,42800.0,-0.34,-144.736084,0.010408,149.526130
9,AURA,42800.0,100.34,42944.736084,13.555403,219.778814
